In [13]:
entrenamiento = 0.6
alpha = 0.1
func_act = "sigmoid"
func_costo = "ECM"
#en costo poner 'ECM' o 'EC_binaria' o 'EC'

In [14]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics, model_selection

import time
#from matplotlib import pyplot as plt
from matplotlib import pylab as plt
from IPython import display

from grafica import *
from ClassRNMulticlase import RNMulticlase

In [15]:
datos = pd.read_csv('../../Datos/Vinos.csv', sep=";")
datos

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [16]:
mapeo = {
    "Class": {1:1, 2:0, 3:0}
}
datos.replace(mapeo,inplace=True)

In [12]:
testeo = 1 - entrenamiento

X = np.array(datos.iloc[:, 1:])
T = np.array(datos['Class'])

X_train, X_test, T_train, T_test = model_selection.train_test_split(
    X, T, test_size=testeo)

normalizador= preprocessing.StandardScaler() #media y desvío
X_train = normalizador.fit_transform(X_train)
X_test  = normalizador.transform(X_test)

In [19]:
if (func_act == 'tanh'):
    T_train = 2*T_train-1
    T_test  = 2*T_test-1

rn = RNMulticlase(alpha=alpha, n_iter=400, cotaE=1e-06, FUN=func_act, COSTO=func_costo, random_state=None)
resul = rn.fit(X_train, T_train)

IndexError: tuple index out of range